Python Analysis and Feature Engineering
=======


This lab will introduce to practical code for two concepts

1. SQL style querying in Pandas 
1. SQL style data shaping in Pandas





**Notebook Setup**

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
df = pd.read_csv('labtrain1.csv')
df.info()

In [ ]:
df.describe()

**Data Setup**

SQL style querying in Pandas
----
1. SELECT or Vertical filters
1. WHERE or Horizontal filters
1. ORDER BY or Sort
1. TOP NN/ LIMIT NN
1. GROUP BY and aggregate samples
1. HAVING or filtered aggregation

**SELECT**

In [ ]:
#SELECT Calendar_Year,NVVar1 from df
#by label
#select_df1 = df[: ,["Calendar_Year","NVVar2"]]
select_df1 = df[["Calendar_Year","NVVar2"]]
select_df1

In [ ]:
collist = ",".join(list(select_df1.columns)).replace(",","\",\"")

In [ ]:
collist

In [ ]:
#SELECT ROW_ID, Calendar_Year, MOdel_year, NVVar1 from df WHERE Calendar_Year = 2005
# by position
select_df2 = df[df.Calendar_Year == 2005].iloc[:,[0,1,2,3]]
select_df2

In [ ]:
col_len = len(select_df2.columns)

In [ ]:
#SELECT COUNT(*) FROM select_df2
row_len = len(select_df2)

In [ ]:
row_len * col_len

In [ ]:
#SELECT COUNT(*) FROM df  WHERE Calendar_Year = 2005
len(df[df.Calendar_Year == 2005])

In [ ]:
for dt in select_df2.dtypes: print(dt)

In [ ]:
#select only numerics
#numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics = [ 'int64']
#df_numeric = df.select_dtypes(include=numerics) 
df.describe(include=numerics).T

In [ ]:
#descriptive stats for strings
df.describe(include="O")

**ORDER BY**

In [ ]:
#SELECT 
#     Calendar_Year,NVVar1 
#FROM
#     df 
#ORDER BY Calendar_Year ASC
select_df1.sort_values('Calendar_Year')

In [ ]:
#SELECT 
#     Calendar_Year,NVVar1 

#FROM
#     df 
#ORDER BY Calendar_Year DESC
select_df1.sort_values('Calendar_Year',ascending=False)

**TOP / LIMIT**

In [ ]:
#SELECT TOP 50 
#Calendar_Year,NVVar1 
#FROM
#     df 
#ORDER BY Calendar_Year DESC
#       or
#SELECT 
#Calendar_Year,NVVar1 
#FROM
#     df 
#ORDER BY Calendar_Year DESC
#LIMIT 50
select_df1.sort_values('Calendar_Year',ascending=False)[0:50]

In [ ]:
#SELECT 
#TOP 50 PERCENT
#Calendar_Year,NVVar1 
#FROM
#     df 
#ORDER BY Calendar_Year DESC
print("before:",len(select_df1))
#select_df1_after = select_df1.sort_values('Calendar_Year',ascending=False).head(int(df.shape[0]*.5))
select_df1_after = (
    select_df1.sort_values('Calendar_Year',ascending=False) 
    .head(int(len(select_df1)*.5)))
print("after:",len(select_df1_after))

In [ ]:
# WITH
#     foo as
#     (
#         SELECT 
#              *
#         from 
#             df
#         ORDER By Calendar_Year
#         LIMIT 50
#     )
#     SELECT 
#         Top 50% *
#     FROM
#         foo
print("before:",len(select_df1))
#select_df3 =select_df1[0:49].sort_values('Calendar_Year',ascending=False)
select_df2_after = \
    select_df1.sort_values(
        #get 50 rows after sort
        'Calendar_Year',ascending=False)[0:49].head( \
            int(select_df1[0:50].sort_values(
                #shape = convert dataframe from query into index = 0 list of rows
                'Calendar_Year',ascending=False).shape[0]*.5))
print("after:",len(select_df2_after))

In [ ]:
col_list = list(set(col if 'NV' in col else None for col in df.columns))
print(col_list)
col_list.pop(3)
col_list


In [ ]:
len(df)

**GROUP BY**

In [ ]:
#SELECT Calendar_Year, count(*) from df group by Calendar_Year
df.Calendar_Year.value_counts()

In [ ]:
#with (sum as select count(*) sum from df)
# ,dtl as (SELECT Calendar_Year, count(*) dtl from df group by Calendar_Year)
#select dtl/sum ...
df.Calendar_Year.value_counts(normalize=True)

In [ ]:
#SELECT Calendar_Year, AVERAGE(NVVar1)) from df group by Calendar_Year
df.groupby('Calendar_Year').mean()['NVVar1']

In [ ]:
# SELECT
#     Calendar_Year
#     ,COUNT(NVVar1) AS 'Calendar_Year_Count'
#     ,COUNT(DISTINCT NVVar1) AS 'Calendar_Year_Count_Distinct'
#     ,AVG(NVVar1) AS 'Calendar_Year_Count'
#     ,STDEVP(NVVar1) AS 'Calendar_Year_Count'
#     ,MIN(NVVar1) AS 'Calendar_Year_Count'
#     ,MAX(NVVar1) AS 'Calendar_Year_Count'
# FROM
#     df
# GROUP BY Calendar_Year
df_agg1 = pd.DataFrame(
    {
        # SELECT COUNT(NVVar1) GROUP BY Calendar_Year
        'Calendar_Year_Count':
            df.groupby('Calendar_Year')
                .count()['NVVar1']
        , 
        #SELECT COUNT(DISTINCT NVVar1) GROUP BY Calendar_Year
        'Calendar_Year_NVVar1_Count_Distinct':
            df.groupby('Calendar_Year')
                ['NVVar1'].nunique()
        ,
        # SELECT AVERAGE(NVVar1) GROUP BY Calendar_Year
        'Calendar_Year_NVVar1_Avg':
            df.groupby('Calendar_Year')
                .mean()['NVVar1']
        , 
                # SELECT STDEV.P(NVVar1) GROUP BY Calendar_Year
        'Calendar_Year_NVVar1_Std':
            df.groupby('Calendar_Year')
                .std()['NVVar1']
        , 
                # SELECT MIN(NVVar1) GROUP BY Calendar_Year
        'Calendar_Year_NVVar1_Min':
            df.groupby('Calendar_Year')
                .min()['NVVar1']        
        , 
                # SELECT MAX(NVVar1) GROUP BY Calendar_Year
        'Calendar_Year_NVVar1_Max':
            df.groupby('Calendar_Year')
                .max()['NVVar1']                
    }
)
df_agg1

In [ ]:
#HasLoss by ModelYear
df_agg2 = pd.DataFrame(
    {
        '2005-2006 Loss Count':
            df.loc[(df.Calendar_Year <= 2006)  ].groupby('Model_Year')
                .sum()['HasLoss']
# outer join between aggregate groupings ( tuple)
,
        '2007-2008 Loss Count':
            df.loc[(df.Calendar_Year > 2006)  ].groupby('Model_Year')
                .sum()['HasLoss']
    }
)
df_agg2

In [ ]:
dfg =df.groupby('Calendar_Year')
type(dfg)

In [ ]:
# SELECT Calendar_Year, SUM(HasLoss) GROUP BY Calendar_Year
dfg.sum()['HasLoss']

In [ ]:
 # SELECT Calendar_Year, COUNT(ROW_ID)... GROUP BY Calendar_Year
 df.groupby('Calendar_Year').count()

SQL style data shaping in Pandas
----
1. AS or derived column review
1. JOIN or key based
1. UNION or schema based
1. Pandas operations that don't ft in SQL Box

**AS or derived column review**

In [ ]:
# WITH 
#     NVVar1_Std as
#         (
#             SELECT
#                 STDEVP(NVVar1) AS NVVar1_Std
#             FROM 
#                 df
#         )
#     NVVar1_Avg as
#         (
#             SELECT
#                 AVG(NVVar1) AS NVVar1_Avg
#             FROM 
#                 df
#         )
# SELECT
#     Calendar_Year
#     ,NVVar1
#     ,
#         (
#             NVVar1
#             -
#             NVVar1_Avg.NVVar1_Avg
#         )
#         /
#         NVVar1_Std.NVVar1_Std
#     AS NVVar1_Z_Score
# FROM
#     df
#     ,NVVar1_Std
#     ,NVVar1_Avg
# persisting in SQl required scalar UDF and view
df['NVVar1_ZScore']                         \
    = (
            df['NVVar1']                    \
            -                               \
            df['NVVar1'].mean()             \
        )                                   \
            /                               \
        df['NVVar1'].std(ddof=0)
#alternately 
#df['NVVar1_ZScore2']= stats.zscore(df['NVVar1'], axis=1, ddof=10.to_frame()
df[["Calendar_Year","NVVar1","NVVar1_ZScore"]].describe()

In [ ]:
from scipy import stats

df["newz"] = stats.zscore(df["NVVar1"])
# T = transpose
df.describe().T

In [ ]:
type(df)

In [ ]:
dfg = (df['NVVar1'] - df['NVVar1'].mean() )/df['NVVar1'].std(ddof=0)
type(dfg)

**JOIN or key based**

In [ ]:
#WITH tbl_cal AS 
# (
#SELECT
# <col> as '<col>',
# sum(1) as 'count',
# mean(<col>) as 'mean',
# stdev(<col>) as 'std',
# min(<col>) as 'min',
# PERCENTILE_DISC(0.25) OVER (PARTITION BY 1 ORDER BY <col>) as '25',
# PERCENTILE_DISC(0.50) OVER (PARTITION BY 1 ORDER BY <col>) as '50',
# PERCENTILE_DISC(0.75) OVER (PARTITION BY 1 ORDER BY <col>) as '75',
# max(<col>) as 'max'
# ...
# SELECT tbl1...
# PIVOT...

newdf = pd.read_csv('labtrain2.csv')
newdf.describe()

In [ ]:
#SELECT a.*,b.* from df a inner JOIN newdf b on a.Calendar_Year = b.NVVar1 
#SELECT a.*,b.* from df a left JOIN newdf b on a.Calendar_Year = b.NVVar1 
#SELECT a.*,b.* from df a right JOIN newdf b on a.Calendar_Year = b.NVVar1 
#SELECT a.*,b.* from df a outer JOIN newdf b on a.Calendar_Year = b.NVVar1 
innerjoin=df.join(newdf, how="inner", lsuffix ='Calendar_Year', rsuffix ='NVVar1')
leftjoin=df.join(newdf, how="left", lsuffix ='Calendar_Year', rsuffix ='NVVar1')
righjoin=df.join(newdf, how="right", lsuffix ='Calendar_Year', rsuffix ='NVVar1')
outerjoin=df.join(newdf, how="outer", lsuffix ='Calendar_Year', rsuffix ='NVVar1')
print("rows df: ",len(df))
print("rows newdf: ",len(newdf))
print("rows innerjoin: ",len(innerjoin))
print("rows leftjoin: ",len(leftjoin))
print("rows rightjoin: ",len(righjoin))
print("rows outerjoin: ",len(outerjoin))
outerjoin

In [ ]:
df.join?

**UNION or schema based**

In [ ]:
#SELECT *,NULL AS Unnamed: 0,	NULL AS NVVar2_ZScore, NULL AS	NVVar3_ZScore	NVVar4_ZScore from df
#UNION ALL
#SELECT * from newdf
df_union= pd.concat([df,newdf],ignore_index=True,sort=False)
print("rows df_union all: ",len(df_union))
df_union

In [ ]:
#SELECT *,NULL AS Unnamed: 0,	NULL AS NVVar2_ZScore, NULL AS	NVVar3_ZScore	NVVar4_ZScore from df
#UNION
#SELECT * from newdf
df_union= pd.concat([df,newdf],ignore_index=True,sort=False)
df_union.drop_duplicates(inplace=True)
print("rows df_union: ",len(df_union))

In [ ]:
df_union.info()

**Pandas operations that don't fit in SQL Box**

In [ ]:
#Where SQL ends and Python begins :)
df.info()
df.groupby(
    ['Calendar_Year']
).agg(
    [
        'min'
        ,'max'
        ,'count'
        ,'nunique'
    ]
)


In [ ]:
df.Calendar_Year.value_counts(normalize=True)

In [ ]:
#hot encoding or case statements forever
df['Calendar_Year_Enum'] = df['Calendar_Year'].astype(str)
df_dummy = pd.get_dummies(df)
df_dummy.describe().T

In [ ]:
df['Calendar_Year_Enum'].describe(include="O")

In [ ]:
9/3*(1+2)

In [ ]:
df.loc[(df.Calendar_Year <= 2006)  ]